# Benchmarking tflite models  

This notebook parses the benchmarking.txt files generated by: 

 [https://www.tensorflow.org/lite/performance/measurement](https://www.tensorflow.org/lite/performance/measurement)  

 TensorFlow Lite benchmark tools currently measure and calculate statistics for the following important performance metrics:

- Initialization time
- Inference time of warmup state
- Inference time of steady state
- Memory usage during initialization time
- Overall memory usage

In [154]:
import os, sys, math, datetime
import pathlib
from pathlib import Path

# import workbench.config.config
from workbench.config.config import initialize
from workbench.utils.utils import create_filepaths
from workbench.tflite_benchmarking import get_index_of_model_profiling_lines
from workbench.wandb import wandb_model_DB, make_clickable, get_architecture_from_model_DB_run_id, get_model_DB_run_id_from_architecture
from workbench.tflite_profiling import get_peak_memory_df, get_tensor_details_df

%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [155]:
import re
from matplotlib import pyplot as plt
#import plotly.express as px
import pandas as pd


# enable plotly in VS Studio Code
#import plotly.io as pio
#pio.renderers.default = "notebook_connected"
#pio.renderers.default = "plotly_mimetype+notebook"

import wandb

In [156]:
# Configure pandas to show all columns & rows
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [157]:
models_dir = initialize()

In [159]:
automated = False
id = wandb.sdk.lib.runid.generate_id() 

global model_name

model_name = "mobilenettiny_0.7_96_c3_o2_l5ll16pl64b0.25.GAPR6"


models_path, models_summary_path, models_image_path, models_layer_df_path, models_tf_path, models_tflite_path, models_tflite_opt_path = create_filepaths(model_name)

/mnt/c/tiny_mlc/tiny_cnn/models


In [160]:
models_benchmark_path = models_dir.joinpath(model_name, f"{model_name}_benchmark.txt")
models_benchmark_path

PosixPath('/mnt/c/tiny_mlc/tiny_cnn/models/mobilenettiny_0.7_96_c3_o2_l5ll16pl64b0.25.GAPR6/mobilenettiny_0.7_96_c3_o2_l5ll16pl64b0.25.GAPR6_benchmark.txt')

In [161]:
#! explorer $models_benchmark_path

# Helper functions

In [162]:
def clean_model_summary(filepath): 

    if not filepath.is_file():
        print(f"This file does not exist: {filepath}")
        return None

    else:
        clean_lines = []
        # Parse the MLTK model summary to grab important metrics   
        with open(filepath, "r", encoding="latin-1") as f:
            lines = f.readlines() # list containing lines of file
            for line in lines:
                line = line.strip() # remove leading/trailing white spaces
                if len(line)> 0:
                    
                    clean_lines.append(line)
                else:
                    pass
            #columns = [] # To store column names
        return clean_lines

In [163]:
def clean_column_names(df):
    cols = df.columns

    clean_cols = []
    for col in cols:
        col = col.strip()
        col = col.replace("[" , "")
        col = col.replace("]" , "")   
        clean_cols.append(col)
        
    return clean_cols

In [164]:
def string_percent_to_float(x):
    x = str(x).strip("%")
    return float(x)

In [165]:
def remove_tabs(text):
    """removes tabs from a list of strings

    Args:
        text (list(str)): list of strings that contains tabs

    Returns:
        list(str): list of strings without tabs
    """
    split_text= []
    for l in text:
        split_text.append((l.split("\t")))
        
    return split_text

# Parsing the model file

if models_benchmark_path.is_file():
    

In [166]:
lines = clean_model_summary(models_benchmark_path)
lines

['STARTING!',
 'Log parameter values verbosely: [0]',
 'Num threads: [1]',
 'Graph: [/mnt/c/tiny_mlc/tiny_cnn/models/mobilenettiny_0.7_96_c3_o2_l5ll16pl64b0.25.GAPR6/mobilenettiny_0.7_96_c3_o2_l5ll16pl64b0.25.GAPR6_INT8.tflite]',
 'Enable op profiling: [1]',
 '#threads used for CPU inference: [1]',
 'Loaded model /mnt/c/tiny_mlc/tiny_cnn/models/mobilenettiny_0.7_96_c3_o2_l5ll16pl64b0.25.GAPR6/mobilenettiny_0.7_96_c3_o2_l5ll16pl64b0.25.GAPR6_INT8.tflite',
 'The input model file size (MB): 0.24476',
 'Initialized session in 163.871ms.',
 'Running benchmark for at least 1 iterations and at least 0.5 seconds but terminate if exceeding 150 seconds.',
 'count=401 first=12645 curr=1208 min=927 max=12645 avg=1246.04 std=615',
 'Running benchmark for at least 50 iterations and at least 1 seconds but terminate if exceeding 150 seconds.',
 'count=778 first=1263 curr=1305 min=976 max=2689 avg=1260.15 std=264',
 'Inference timings in us: Init: 163871, First inference: 12645, Warmup (avg): 1246.04, 

In [167]:
def get_profiling_lines(lines):
    #print(lines)
    # find section split
    for i, line in enumerate(lines):
        #print(f"{i} : {line}")
        #if line.contains('Operator-wise'):
        if ("Operator-wise" in line):
            split_line = i
            print(f"{i} - profiling")
            #print(f"Splitting text at line {split_line}")
            model_profiling = lines[:split_line]
            operator_profiling = lines[split_line+1:]
            return model_profiling, operator_profiling
        else:
            pass
    print("WARNING: Could not find the text: 'Operator-wise Profiling Info for Regular Benchmark Runs:'")
    return (None, None)

In [168]:
model_profiling, operator_profiling = get_profiling_lines(lines)

33 - profiling


In [169]:
model_lines_dict = get_index_of_model_profiling_lines(model_profiling)

In [170]:
def get_model_profiling_stats(model_profiling):
    model_lines_dict = get_index_of_model_profiling_lines(model_profiling)

    model_stats = {}

    # extract model size
    model_size_string = model_profiling[model_lines_dict["model_file_size"]]
    model_size_string
    model_size= float(model_size_string.split(":")[-1].strip())
    model_stats["model_size_MB"] = model_size

    # extract inference times
    inference_string = model_profiling[model_lines_dict["inference_timings"]]
    #print(inference_string)
    inference_strings = inference_string.split(",")
    for item in inference_strings:
        item_time = item.split()[-1]
        #print(item_time)

    model_stats["init_us"] = int(inference_strings[0].split()[-1].strip())
    model_stats["first_inference_us"] = int(inference_strings[1].split()[-1].strip())
    model_stats["warmup_avg_us"] = float(inference_strings[2].split()[-1].strip())
    model_stats["inference_avg_us"] = float(inference_strings[3].split()[-1].strip())

    # model initialization timings
    init_string = model_profiling[model_lines_dict["initialization_ms"]]
    model_stats["initialization_ms"] = float(init_string.split()[-1].replace("ms.",""))


    # initialization run order
    init_run_order = model_profiling[model_lines_dict["run_order"]: model_lines_dict["top_by_computation_time"]]
    init_run_order = remove_tabs(init_run_order)

    model_stats["modify_graph_with_delegate_ms_first"] = float(init_run_order[2][1].strip())
    model_stats["modify_graph_with_delegate_ms_avg"] = float(init_run_order[2][2].strip())
    model_stats["modify_graph_with_delegate_ms_%"] = float(init_run_order[2][3].strip().replace("%", ""))
    model_stats["modify_graph_with_delegate_mem_KB"] = float(init_run_order[2][5].strip())

    model_stats["allocate_tensors_ms_first"] = float(init_run_order[3][1].strip())
    model_stats["allocate_tensors_ms_avg"] = float(init_run_order[3][2].strip())
    model_stats["allocate_tensors_ms_%"] = float(init_run_order[3][3].strip().replace("%", ""))

    return model_stats

In [171]:
model_stats = get_model_profiling_stats(model_profiling)
model_stats

{'model_size_MB': 0.24476,
 'init_us': 163871,
 'first_inference_us': 12645,
 'warmup_avg_us': 1246.04,
 'inference_avg_us': 1260.15,
 'initialization_ms': 163.871,
 'modify_graph_with_delegate_ms_first': 10.882,
 'modify_graph_with_delegate_ms_avg': 10.882,
 'modify_graph_with_delegate_ms_%': 99.725,
 'modify_graph_with_delegate_mem_KB': 856.0,
 'allocate_tensors_ms_first': 0.03,
 'allocate_tensors_ms_avg': 0.03,
 'allocate_tensors_ms_%': 0.275}

In [172]:
def get_profiling_stats_cpu(filepath):
    """
    Parse "bechmarking.txt" file that has been generated from a .tflite model with TensorFlow benchmarking tool.  

    The native benchmark binary for linux is available from this page:  
    [https://www.tensorflow.org/lite/performance/measurement](https://www.tensorflow.org/lite/performance/measurement)   

    Args:
        filepath (pathlib.Path): filepath to the extracted .txt file

    Returns:
        dict: dictionary with extracted model summary profiling information 
    """

    lines = clean_model_summary(filepath)
    model_profiling, operator_profiling = get_profiling_lines(lines)
    profiling_stats = get_model_profiling_stats(model_profiling)
    return profiling_stats


In [173]:
prof_stats = get_profiling_stats_cpu(models_benchmark_path)
prof_stats

33 - profiling


{'model_size_MB': 0.24476,
 'init_us': 163871,
 'first_inference_us': 12645,
 'warmup_avg_us': 1246.04,
 'inference_avg_us': 1260.15,
 'initialization_ms': 163.871,
 'modify_graph_with_delegate_ms_first': 10.882,
 'modify_graph_with_delegate_ms_avg': 10.882,
 'modify_graph_with_delegate_ms_%': 99.725,
 'modify_graph_with_delegate_mem_KB': 856.0,
 'allocate_tensors_ms_first': 0.03,
 'allocate_tensors_ms_avg': 0.03,
 'allocate_tensors_ms_%': 0.275}

# Operator profiling

## Helper Functions

In [174]:
def get_operator_df(text, name=""):
    df = pd.DataFrame(text)
    df.rename(columns=df.iloc[0, :], inplace=True) 
    df.drop(df.index[0], inplace=True)
    df.columns = clean_column_names(df)
    # try:
    df["%"] = df["%"].apply(string_percent_to_float)
    # except:
    #     df["avg %"] = df["avg %"].apply(string_percent_to_float)
    # try:
    df["cdf%"] = df["cdf%"].apply(string_percent_to_float)
    # except:
    #     df["cdf %"] = df["cdf %"].apply(string_percent_to_float)
    df["first"] = df["first"].map(float)
    df["avg ms"] = df["avg ms"].map(float)
    df["mem KB"] = df["mem KB"].map(float)
    df["times called"] = df["times called"].map(int)#
    df.reset_index(inplace=True, drop=False)
    df.rename({"index": "_index"}, inplace=True)
    #df.Name = name

    return df

In [175]:
def get_node_df(text, name=""):
    df = pd.DataFrame(text)
    df.rename(columns=df.iloc[0, :], inplace=True) 
    df.drop(df.index[0], inplace=True)
    df.columns = clean_column_names(df)
    df["avg %"] = df["avg %"].apply(string_percent_to_float)
    df["cdf %"] = df["cdf %"].apply(string_percent_to_float)
    df["avg ms"] = df["avg ms"].map(float)
    df["mem KB"] = df["mem KB"].map(float)
    df["times called"] = df["times called"].map(int)
    df["count"] = df["count"].map(int)
    df.Name = name

    return df

In [176]:
def get_index_of_operator_profiling_lines(lines):
    operator_lines_dict = {}
    for i, line in enumerate(lines):
        #if line.str.contains('= Run Order ='):
        if "= Run Order =" in line:
            operator_lines_dict["run_order"] = i
        elif "= Top by Computation Time =" in line:
            operator_lines_dict["top_by_computation_time"] = i
        elif "= Summary by node type =" in line:
            operator_lines_dict["summary_by_node_type"] = i

    # print(operator_lines_dict)
    return operator_lines_dict

In [178]:
def get_summary_by_node_type_df(operator_profiling, operator_lines_dict, model_name):
    summary_by_node_type = operator_profiling[operator_lines_dict["summary_by_node_type"] +1:-3]
    summary_by_node_type = remove_tabs(summary_by_node_type)
    summary_by_node_type_df = get_node_df(summary_by_node_type, name=f"Summary by node type - {model_name}")
    return summary_by_node_type_df

In [179]:
def get_operator_top_by_comp_time_df(operator_profiling, operator_lines_dict, model_name):

    top_by_comp_time = operator_profiling[
        operator_lines_dict["top_by_computation_time"]+ 1 : operator_lines_dict["summary_by_node_type"]- 1]
    top_by_comp_time_clean = remove_tabs(top_by_comp_time)

    df = pd.DataFrame(top_by_comp_time_clean)

    df.rename(columns=df.iloc[0, :], inplace=True)
    df.drop(df.index[0], inplace=True)
    df.columns = clean_column_names(df)

    df["%"] = df["%"].apply(string_percent_to_float)
    df["cdf%"] = df["cdf%"].apply(string_percent_to_float)
    df["first"] = df["first"].map(float)
    df["avg ms"] = df["avg ms"].map(float)
    df["mem KB"] = df["mem KB"].map(float)
    df["times called"] = df["times called"].map(int)

    return df

In [180]:
def get_run_order_df(operator_lines_dict, operator_profiling, model_name):
    run_order = operator_profiling[operator_lines_dict["run_order"] +1: operator_lines_dict["top_by_computation_time"]]
    run_order = remove_tabs(run_order)
    df = get_operator_df(run_order, name= f"Run order - {model_name}")
    return df

In [181]:
def get_profiling_dataframes_cpu(filepath, model_name):
    """
    Parse "bechmarking.txt" file that has been generated from a .tflite model with TensorFlow benchmarking tool.  

    The native benchmark binary for linux is available from this page:  
    [https://www.tensorflow.org/lite/performance/measurement](https://www.tensorflow.org/lite/performance/measurement)   

    Args:
        filepath (pathlib.Path): filepath to the extracted .txt file

    Returns:
        list(pd.DataFrame, pd.DataFrame, pd.DataFrame): tuple containing the following dataframes:

            operator_run_order_df, operator_by_comp_time_df, summary_node_type_df
    """

    lines = clean_model_summary(filepath)
    model_profiling, operator_profiling = get_profiling_lines(lines)
    operator_lines_dict = get_index_of_operator_profiling_lines(operator_profiling)

    operator_run_order_df = get_run_order_df(operator_lines_dict, operator_profiling, model_name)
    operator_by_comp_time_df = get_operator_top_by_comp_time_df(operator_profiling, operator_lines_dict, model_name)
    summary_node_type_df = get_summary_by_node_type_df(operator_profiling, operator_lines_dict, model_name)

    return operator_run_order_df, operator_by_comp_time_df, summary_node_type_df

# Create profiling dataframes

In [182]:
operator_run_order_df, operator_by_comp_time_df, summary_node_type_df = get_profiling_dataframes_cpu(models_benchmark_path, model_name)

33 - profiling


In [183]:
operator_run_order_df

,index,node type,first,avg ms,%,cdf%,mem KB,times called,Name
0,1,"Convolution (NHWC, QC8) IGEMM",0.176,0.159,12.974,12.974,0.0,1,"Delegate/Convolution (NHWC, QC8) IGEMM:0"
1,2,"Convolution (NHWC, QC8) DWConv",0.101,0.107,8.729,21.703,0.0,1,"Delegate/Convolution (NHWC, QC8) DWConv:1"
2,3,"Convolution (NHWC, QC8) GEMM",0.124,0.131,10.704,32.407,0.0,1,"Delegate/Convolution (NHWC, QC8) GEMM:2"
3,4,"Convolution (NHWC, QC8) DWConv",0.045,0.048,3.930,36.337,0.0,1,"Delegate/Convolution (NHWC, QC8) DWConv:3"
4,5,"Convolution (NHWC, QC8) GEMM",0.095,0.099,8.074,44.411,0.0,1,"Delegate/Convolution (NHWC, QC8) GEMM:4"
5,6,"Convolution (NHWC, QC8) DWConv",0.087,0.091,7.435,51.846,0.0,1,"Delegate/Convolution (NHWC, QC8) DWConv:5"
6,7,"Convolution (NHWC, QC8) GEMM",0.153,0.160,13.040,64.886,0.0,1,"Delegate/Convolution (NHWC, QC8) GEMM:6"
7,8,"Convolution (NHWC, QC8) DWConv",0.022,0.024,1.946,66.832,0.0,1,"Delegate/Convolution (NHWC, QC8) DWConv:7"
8,9,"Convolution (NHWC, QC8) GEMM",0.073,0.078,6.339,73.171,0.0,1,"Delegate/Convolution (NHWC, QC8) GEMM:8"
9,10,"Convolution (NHWC, QC8) DWConv",0.042,0.044,3.544,76.715,0.0,1,"Delegate/Convolution (NHWC, QC8) DWConv:9"


In [184]:
operator_by_comp_time_df

,node type,first,avg ms,%,cdf%,mem KB,times called,Name
1,"Convolution (NHWC, QC8) GEMM",0.153,0.160,13.040,13.040,0.0,1,"Delegate/Convolution (NHWC, QC8) GEMM:6"
2,"Convolution (NHWC, QC8) IGEMM",0.176,0.159,12.974,26.014,0.0,1,"Delegate/Convolution (NHWC, QC8) IGEMM:0"
3,"Convolution (NHWC, QC8) GEMM",0.128,0.137,11.168,37.182,0.0,1,"Delegate/Convolution (NHWC, QC8) GEMM:10"
4,"Convolution (NHWC, QC8) GEMM",0.124,0.131,10.704,47.886,0.0,1,"Delegate/Convolution (NHWC, QC8) GEMM:2"
5,"Convolution (NHWC, QC8) DWConv",0.101,0.107,8.729,56.615,0.0,1,"Delegate/Convolution (NHWC, QC8) DWConv:1"
6,"Convolution (NHWC, QC8) GEMM",0.095,0.099,8.074,64.688,0.0,1,"Delegate/Convolution (NHWC, QC8) GEMM:4"
7,"Convolution (NHWC, QC8) DWConv",0.087,0.091,7.435,72.123,0.0,1,"Delegate/Convolution (NHWC, QC8) DWConv:5"
8,"Convolution (NHWC, QC8) GEMM",0.073,0.078,6.339,78.463,0.0,1,"Delegate/Convolution (NHWC, QC8) GEMM:8"
9,"Convolution (NHWC, QC8) DWConv",0.045,0.048,3.930,82.393,0.0,1,"Delegate/Convolution (NHWC, QC8) DWConv:3"
10,"Convolution (NHWC, QC8) DWConv",0.042,0.044,3.544,85.937,0.0,1,"Delegate/Convolution (NHWC, QC8) DWConv:9"


In [185]:
summary_node_type_df

,Node type,count,avg ms,avg %,cdf %,mem KB,times called
1,"Convolution (NHWC, QC8) GEMM",14,0.696,57.190,57.190,0.0,14
2,"Convolution (NHWC, QC8) DWConv",13,0.360,29.581,86.771,0.0,13
3,"Convolution (NHWC, QC8) IGEMM",1,0.159,13.065,99.836,0.0,1
4,SOFTMAX,1,0.001,0.082,99.918,0.0,1
5,RESHAPE,1,0.001,0.082,100.000,0.0,1
6,"Global Average Pooling (NWC, QS8)",1,0.000,0.000,100.000,0.0,1


# Peak memory usage

In [197]:
models_peak_memory_path = models_dir.joinpath(model_name, f"{model_name}_peak_memory")
model_peak_memory_schedule = models_dir.joinpath(model_name, f"{model_name}_peak_memory", "execution_schedule_info.csv")
peak_memory_tensor_details = models_dir.joinpath(model_name, f"{model_name}_peak_memory", "tensor_details.csv")
peak_memory_img = models_dir.joinpath(model_name, f"{model_name}_peak_memory", "peak_memory.png")
models_peak_memory_path

PosixPath('/mnt/c/tiny_mlc/tiny_cnn/models/mobilenettiny_0.7_96_c3_o2_l5ll16pl64b0.25.GAPR6/mobilenettiny_0.7_96_c3_o2_l5ll16pl64b0.25.GAPR6_peak_memory')

In [198]:
models_tflite_opt_path

PosixPath('/mnt/c/tiny_mlc/tiny_cnn/models/mobilenettiny_0.7_96_c3_o2_l5ll16pl64b0.25.GAPR6/mobilenettiny_0.7_96_c3_o2_l5ll16pl64b0.25.GAPR6_INT8.tflite')

In [199]:
tflite_tool_path = Path.cwd().parent.joinpath("tflite-tools", "tflite_tools.py")
tflite_tool_path

PosixPath('/mnt/c/tiny_mlc/tflite-tools/tflite_tools.py')

In [200]:
! python $tflite_tool_path -i $models_tflite_opt_path --csv $models_peak_memory_path

Writing model analysis to /mnt/c/tiny_mlc/tiny_cnn/models/mobilenettiny_0.7_96_c3_o2_l5ll16pl64b0.25.GAPR6/mobilenettiny_0.7_96_c3_o2_l5ll16pl64b0.25.GAPR6_peak_memory in CSV format


In [201]:
! python $tflite_tool_path -i $models_tflite_opt_path --plot $peak_memory_img

Tensor information (weights excluded):
+----+----------------------------------------------------------------------------------+------------------+-----------------+
| Id |                                      Tensor                                      |      Shape       | Size in RAM (B) |
+----+----------------------------------------------------------------------------------+------------------+-----------------+
|  0 |                            serving_default_input_1:0                             |  (1, 96, 96, 3)  |          27,648 |
| 59 | mobilenettiny/re_lu/Relu6;mobilenettin...chNormV3;mobilenettiny/conv2d_84/Conv2D | (1, 48, 48, 22)  |          50,688 |
| 60 | mobilenettiny/re_lu_1/Relu6;mobilenett...bilenettiny/depthwise_conv2d_64/BiasAdd | (1, 48, 48, 22)  |          50,688 |
| 61 | mobilenettiny/re_lu_2/Relu6;mobilenett...chNormV3;mobilenettiny/conv2d_85/Conv2D | (1, 48, 48, 44)  |         101,376 |
| 62 | mobilenettiny/re_lu_3/Relu6;mobilenett...atch_normalization_108/F

In [92]:
#! python $tflite_tool_path -i $models_tf_path

In [99]:
tensor_details_df = get_tensor_details_df(peak_memory_tensor_details)
tensor_details_df.head()

Reading in /mnt/c/tiny_mlc/tiny_cnn/models/mininet_0.1_96_c3_o2_000/mininet_0.1_96_c3_o2_000_peak_memory/tensor_details.csv
Cleaning up the dataframe.


,index,id,layer_name,shape,size_b,name_long
0,0,0,serving_default_input_1:0,"(1, 96, 96, 3)",27648,serving_default_input_1:0
1,1,7,tfl.quantize,"(1, 96, 96, 3)",27648,tfl.quantize
2,2,8,mininet/conv2d_84/Conv2D,"(1, 48, 48, 3)",6912,mininet/re_lu/Relu;mininet/batch_normalization_83/FusedBatchNormV3;mininet/batch_normalization_83/FusedBatchNormV3/ReadVariableOp;mininet/conv2d_84/BiasAdd;mininet/conv2d_84/Conv2D
3,3,9,mininet/global_average_pooling2d_17/Mean,"(1, 1, 1, 3)",3,global_average_pooling2d_17
4,4,10,mininet/conv2d_85/Conv2D,"(1, 1, 1, 2)",2,mininet/conv2d_85/BiasAdd;mininet/conv2d_85/BiasAdd/ReadVariableOp;mininet/conv2d_85/Conv2D


## Peak memory df

In [103]:
peak_memory_df = get_peak_memory_df(model_peak_memory_schedule)
peak_memory_df.head()

Reading in /mnt/c/tiny_mlc/tiny_cnn/models/mininet_0.1_96_c3_o2_000/mininet_0.1_96_c3_o2_000_peak_memory/execution_schedule_info.csv
Cleaning up the dataframe.


,index,layer_name,tensor_IDs,RAM_b,operator
0,0,tfl.quantize,"(0, 7)",55296,tfl.quantize
1,1,conv2d_84,"(7, 8)",34560,mininet/re_lu/Relu;mininet/batch_normalization_83/FusedBatchNormV3;mininet/batch_normalization_83/FusedBatchNormV3/ReadVariableOp;mininet/conv2d_84/BiasAdd;mininet/conv2d_84/Conv2D
2,2,global_average_pooling2d_17,"(8, 9)",6915,mininet/global_average_pooling2d_17/Mean
3,3,conv2d_85,"(9, 10)",5,mininet/conv2d_85/BiasAdd;mininet/conv2d_85/BiasAdd/ReadVariableOp;mininet/conv2d_85/Conv2D
4,4,flatten,"(11, 10)",4,mininet/flatten/Reshape


In [104]:
peak_memory_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   index       7 non-null      int64 
 1   layer_name  7 non-null      object
 2   tensor_IDs  7 non-null      object
 3   RAM_b       7 non-null      int64 
 4   operator    7 non-null      object
dtypes: int64(2), object(3)
memory usage: 408.0+ bytes


# Logging to wandb

In [102]:
PROJECT = "model_DB"

run = wandb.init(
        project=PROJECT, 
        name = model_name,
        id = id, 
        resume="allow",
        sync_tensorboard=True
        )

# Specify the configuration variables
config = wandb.config
config.architecture = model_name

run.log(model_stats)
run.log(prof_stats)

# # Create a table
operator_top_by_time_table = wandb.Table(dataframe=operator_by_comp_time_df)

summary_by_node_type_table = wandb.Table(dataframe=summary_node_type_df)

operator_run_order_table = wandb.Table(dataframe=operator_run_order_df)

run.log({"Profiling operators - top by time": operator_top_by_time_table})
run.log({"Node type summary" : summary_by_node_type_table})
run.log({"Run order" : operator_run_order_table})

wandb.finish()

allocate_tensors_ms_%,▁▁
allocate_tensors_ms_avg,▁▁
allocate_tensors_ms_first,▁▁
first_inference_us,▁▁
inference_avg_us,▁▁
init_us,▁▁
initialization_ms,▁▁
model_size_MB,▁▁
modify_graph_with_delegate_mem_KB,▁▁
modify_graph_with_delegate_ms_%,▁▁
modify_graph_with_delegate_ms_avg,▁▁
